In [ ]:
import os
import numpy as np
import pandas as pd
import joblib
import matplotlib.pyplot as plt
import matplotlib as mpl

mpl.rcParams['font.size'] = 10
mpl.rcParams['font.family'] = 'sans-serif'

# Week 1 & 2

In [ ]:
df_feat_com = pd.read_csv("../data/features/stoich120_fingerprints.csv")
df_feat_ofm = pd.read_csv("../data/features/ofm_fingerprints.csv")
df_label = pd.read_csv("../data/qmof.csv")

## Discussion 1

- How many structures are there in the database?
- What is the dimension of the feature vectores?

Hint: call (`df.shape`)[https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.shape.html]

In [ ]:
#fillme

## Discussion 2

- Are there any features that have the same values for all structures? If yes, which are they?
- What kind of effects will they have on the model performance?

In [ ]:
#fillme
com_counting = df_feat_com.describe()
const_feat = [key for key in com_counting.keys() if com_counting[key]["std"]==0]

In [ ]:
const_feat

In [ ]:
uni_feat = [key for key in df_feat_com.keys() if key not in const_feat]

In [ ]:
df_feat_com_uni = df_feat_com[uni_feat]

In [ ]:
df_feat_com_uni.to_csv("../data/features/stoich120_fingerprints_processed.csv", index=False)

- Model with original fingerprints \
Train size:  16299 \
Test size:  4075 \
Train/test MAE:  0.3820170159136949 0.4528077329818929 \
Train/test MSE:  0.2592329547360367 0.3575536280128822 \
Train/test r2:  0.7984888749541184 0.7152858603480199 \

- Model remove part of features \
Train size:  16299 \
Test size:  4075 \
Train/test MAE:  0.38201701591369436 0.4528077329819106 \
Train/test MSE:  0.25923295473605484 0.3575536280128981 \
Train/test r2:  0.7984888749541043 0.7152858603480073 \

## Check the distribution of band gaps

Hint: `df.describe()`

In [ ]:
#fillme
df_label["outputs.pbe.bandgap"].describe()

In [ ]:
q = df_label["outputs.pbe.bandgap"].quantile(0.75)

In [ ]:
plt.hist(df_label["outputs.pbe.bandgap"], bins=40)
plt.plot([q, q], [0, 1200], 'r--')

# Week 3

## Read true and predicted bandgap

In [ ]:
#fillme
df_train = pd.read_csv("../results/stoich120/train_results.csv")
df_test = pd.read_csv("../results/stoich120/test_results.csv")

In [ ]:
train_min = df_train["ML"].min()
train_max = df_train["ML"].max()

test_min = df_test["ML"].min()
test_max = df_test["ML"].max()

## Plot the parity plot

In [ ]:
fig, axs = plt.subplots(1,2)
fig.set_size_inches((20,9))

for ax in axs:
    ax.set_aspect('equal')
axs[0].scatter(df_train["DFT"], df_train["ML"])
axs[0].plot([train_min, train_max], [train_min, train_max], 'r--')

axs[1].scatter(df_test["DFT"], df_test["ML"])
axs[1].plot([train_min, train_max], [train_min, train_max], 'r--')

#fig.savefig('#fillme')

# Week 4

## Discussion

- What is your hyperparameter searching range? 
- What is the best hyperparameter you found?

In [ ]:
m1_loss = #fillme df.read("../results/stoich120/loss_ft.csv") # Saving loss to csv need to be added in w4-finetune
m2_loss = #fillme df.read("../results/ofm/loss_ft.csv")

In [ ]:
plt.bar(#fillme)

# Week 5

## Plot the learning curve

In [ ]:
# read data

lc_data = #fillme

In [ ]:
# make plots

plt.plot(#fillme)
plt.errorbar(#fillme)

## Use the model to predict band gap of MOF-5

Follow the steps:
All files of MOF-5 needed is in ../data/mof5s

- Featurize MOF-5 with python scripts in ../machine_learning/w1w2-featurizer
- Load saved model
- Predict the band gap of MOF-5
- Compare the model estimated band gap with DFT calculated value

In [ ]:
#fillme